In [1]:
from functions import *
import torch

    
net = Net(3, 6)
T = torch.tensor([1])
dt = torch.tensor([0.01])
S0 = torch.tensor([100])
q = torch.tensor([0.01])
A = torch.tensor([3, 4, 5])
B = torch.tensor([0.01, 0.01, 0.01])
Q = torch.tensor([100])
z = torch.tensor([1, 2, 3])
delta = torch.tensor([0.01])
gamma = torch.tensor([0.01])
sigma = torch.tensor([0.01])

# use cuda if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)
T = T.to(device)
dt = dt.to(device)
S0 = S0.to(device)
q = q.to(device)
A = A.to(device)
B = B.to(device)
Q = Q.to(device)
z = z.to(device)
delta = delta.to(device)
gamma = gamma.to(device)
sigma = sigma.to(device)

In [2]:
policy_net = Net(3, 6)
value_net = Net(3, 1)
policy_net.to(device)
value_net.to(device)

Net(
  (fc1): Linear(in_features=3, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
)

In [3]:
S, buy_orders, sell_orders, q, t, bid_vectors, ask_vectors = Test_Data_Simulation_Stochastic(T, dt, sigma, S0, policy_net, A, B, gamma, z)

In [4]:
probability(policy_net, S, q, t, bid_vectors, ask_vectors, T, dt, B, gamma, z)

tensor([-7.5875, -2.5408, -4.1090, -2.7285, -2.4919, -5.6620, -2.9564, -4.0969,
        -4.5362, -5.1323, -5.5316, -3.0288, -2.6366, -6.3028, -3.4439, -2.9875,
        -5.3125, -4.9088, -6.0951, -2.3368, -4.5408, -5.1248, -3.5844, -6.7986,
        -2.7194, -3.2942, -7.9807, -3.2720, -4.8264, -4.7399, -2.1282, -3.3936,
        -3.6182, -5.3389, -4.5723, -4.5327, -2.9593, -2.8014, -4.0396, -5.4292,
        -4.2116, -2.8150, -4.0691, -3.1878, -5.3028, -5.0630, -4.4935, -4.8417,
        -4.2808, -6.7766, -5.6428, -5.2572, -2.3303, -8.2016, -3.7184, -3.6828,
        -4.8931, -5.7831, -7.7189, -2.9431, -4.2420, -5.1556, -4.3878, -5.3902,
        -3.5326, -4.2669, -2.5768, -3.2487, -5.7830, -5.3301, -4.6436, -5.1412,
        -2.9616, -7.7474, -6.2604, -7.5793, -7.8735, -4.0485, -3.3032, -4.4068,
        -4.9887, -4.1654, -3.8330, -4.6948, -4.5112, -4.7967, -3.9964, -6.8502,
        -4.7706, -6.6547, -3.8645, -4.1646, -4.1440, -3.9500, -4.2391, -5.6295,
        -2.8709, -3.3287, -2.2387,  0.00

In [5]:
policy_loss(policy_net, value_net, S, q, t, buy_orders, sell_orders, bid_vectors, ask_vectors, T, dt, A, B, gamma, delta, z, Q)

tensor(45.0459, device='cuda:0', grad_fn=<DivBackward0>)

In [6]:
def probability(policy_net, S, q, t, buy_orders, sell_orders, T, dt, A, B, gamma, z, Q):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    N = int(T / dt)
    p = torch.zeros(N, device = device)
    for i in range(N - 1):
        mean = policy_net.forward(t[i], S[i], q[i])
        hold1 = z * buy_orders[i]
        hold2 = z * sell_orders[i]
        hold = torch.cat((hold1, hold2), 0)
        diag = torch.cat((gamma / (2 * z * B), gamma / (2 * z * B)), 0)
        cov = torch.diag(diag)
        p[i] = torch.distributions.multivariate_normal.MultivariateNormal(mean, cov).log_prob(hold)
    return p


probability(policy_net, S, q, t, buy_orders, sell_orders, T, dt, A, B, gamma, z, Q)

tensor([-39.2813, -12.5963, -12.5952, -12.5944, -12.5941, -12.5936, -12.5930,
        -12.5924, -12.5915, -12.5910, -18.8366, -12.5005, -12.4999, -12.4991,
        -12.4983, -12.4973, -12.4962, -27.9661, -12.5853, -16.0391, -12.8328,
        -12.8318, -12.8310, -39.3109, -12.5821, -12.5813, -39.2257, -12.4623,
        -12.4615, -12.4610, -18.8155, -12.3986, -12.3982, -12.3975, -12.3970,
        -12.3965, -12.3955, -12.3950, -12.3945, -12.3938, -20.9788, -12.4533,
        -12.4524, -12.4518, -12.4514, -12.4506, -12.4499, -12.4488, -27.9041,
        -16.0547, -12.7214, -12.7212, -15.6497, -13.0343, -21.4311, -13.1255,
        -39.5971, -28.2584, -13.0305, -28.5222, -13.2338, -13.2328, -13.2319,
        -19.1694, -13.0259, -13.0253, -13.0248, -15.3377, -39.8592, -13.0229,
        -13.0220, -17.4501, -17.3330, -12.7960, -12.7952, -12.7950, -12.7944,
        -12.7935, -12.7928, -12.7923, -12.7917, -12.7912, -12.7902, -12.7893,
        -12.7888, -12.7882, -12.7879, -12.7870, -17.1885, -12.70

In [7]:
res = ResNet_Conv(3, 4, 1, 16, 3)
res.to(device)
res.forward(T, S0, q)

ValueError: only one element tensors can be converted to Python scalars

In [ ]:

y = net.forward(T, S0, q)


In [ ]:
y

tensor([-5.6179], device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
S, buy_orders, sell_orders, q, t = Train_Data_Simulation(1, 0.01, 0.1, S0, A, B, Q, z, delta, gamma, net)
total_loss(net, S, q, t, 0.001, buy_orders, sell_orders, z, delta, Q, A, B, gamma)

tensor(697.9450, device='cuda:0', grad_fn=<MulBackward0>)

In [ ]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
for epoch in range(200):
    S, buy_orders, sell_orders, q, t = Train_Data_Simulation(1, 0.01, 0.1, S0, A, B, Q, z, delta, gamma, net)
    loss = total_loss(net, S, q, t, 0.001, buy_orders, sell_orders, z, delta, Q, A, B, gamma)
    print(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

573.9168701171875
758.518310546875
572.6505737304688
550.671875


KeyboardInterrupt: 